QuantumHelpers is a library intended to help with learning and understanding quantum computing, or can be used for quick calculations.

In [1]:
import src.QuantumHelpers as qh
import numpy as np

In [2]:
zeroState = qh.eval("|0><0|")
OneState = qh.eval("|1><1|")


In [3]:
a = qh.eval("|10>")
#cnot = qh.eval("(|0><0|)X + (|1><1|)I")
cnot = qh.QuantumElement(np.array([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]]), qh.WaveFunctionTokens.OPERATOR)
b = cnot * a
a.print()
b.print()

1 |10>
1 |10>


In [4]:
print(qh.eval("-1j(Rz(π))*(Ry(π/2))"))

print(qh.eval("Rz(Pi)"))

print(qh.eval("Ry(2*Pi)"))

[[-1/√2 1/√2]
 [1/√2 1/√2]]
[[0-1j 0]
 [0 0+1j]]
[[-1 0]
 [0 -1]]


In [5]:
print(qh.eval("Exp(((1/2)Z + (1/4)Y) * Pi)"))

[[5.494 0-1.256j]
 [0+1.256j 0.469]]


In [6]:
print(qh.eval("(1/8)|0>"))

[[0.125]
 [0]]


In [15]:
print(qh.eval("-1j * (Pi * 1j / 2) * Z"))
print(qh.eval("Exp(-1j * (Pi * 1j / 2) * Z)"))
print(qh.eval("H * Exp(-1j * (Pi * 1j / 2) * Z) * H"))

[[1.571 0]
 [0 -1.571]]
[[4.810 0]
 [0 0.208]]
[[-0.971 2.877]
 [2.877 0.971]]


In [66]:
print(qh.eval("(-1j*(Rz(π))*(Ry(3 * π/2)))"))

x = qh.eval("(Pi/2)Z + (Pi/4)Y")
print(x)
#x = x * np.pi
print(x)
x = x * -1j
print(x)
x = qh.QuantumElement(qh.exponentiateMatrix(x.data), qh.WaveFunctionTokens.OPERATOR)
print(x)
x = x * -1j
print(x)




[[1/√2 1/√2]
 [1/√2 -1/√2]]
[[1.571 0-0.785j]
 [0+0.785j -1.571]]
[[1.571 0-0.785j]
 [0+0.785j -1.571]]
[[0-1.571j -0.785]
 [0.785 0+1.571j]]
[[-0.184-0.879j -0.440]
 [0.440 -0.184+0.879j]]
[[-0.879+0.184j 0+0.440j]
 [0-0.440j 0.879+0.184j]]


In [69]:
qh.eval("((Rx(Pi))(Rx(Pi)))|00>").print()

0 |00>
0 |01>
0 |10>
-1 |11>


In [240]:
full_hp = qh.QuantumElement(np.zeros((2**8, 2**8)), qh.WaveFunctionTokens.OPERATOR)
xs = ['XXIIIIII', 'IXXIIIII', 'IIXXIIII', 'IIIXXIII', 'IIIIXXII', 'IIIIIXXI', 'IIIIIIXX']
ys = ['YYIIIIII', 'IYYIIIII', 'IIYYIIII', 'IIIYYIII', 'IIIIYYII', 'IIIIIYYI', 'IIIIIIYY']
zs = ['ZZIIIIII', 'IZZIIIII', 'IIZZIIII', 'IIIZZIII', 'IIIIZZII', 'IIIIIZZI', 'IIIIIIZZ']
for i in range(7):
    full_hp += qh.eval(xs[i]) + qh.eval(ys[i]) + qh.eval(zs[i])

print(full_hp)

eigs, eigVecs = np.linalg.eig(full_hp.data)
print(np.min(eigs))

print(np.sum(full_hp.data * np.transpose(eigVecs)[np.argmin(eigs)] - np.transpose(eigVecs)[np.argmin(eigs)] * np.min(eigVecs)))

gs_psi = qh.QuantumElement(np.transpose(eigVecs)[np.argmin(eigs)], qh.WaveFunctionTokens.OPERATOR)



[[7.000 0 0 ... 0 0 0]
 [0 5.000 2.000 ... 0 0 0]
 [0 2.000 3.000 ... 0 0 0]
 ...
 [0 0 0 ... 3.000 2.000 0]
 [0 0 0 ... 2.000 5.000 0]
 [0 0 0 ... 0 0 7.000]]
(-13.499730394751591+1.5777218121126138e-30j)
(-2.4311021945555567e-13-7.651568084186147e-30j)


In [241]:
def Hp(a, dt):
    theta = -2 * dt * a
    print(f"iswap with {theta}")
    hpIswap = p_iSwap(theta)
    hprzz = rzz(theta)
    I = qh.eval("I")
    evens = (hpIswap & hpIswap & hpIswap & hpIswap) * (hprzz & hprzz & hprzz & hprzz)
    odds = (I & hpIswap & hpIswap & hpIswap & I) * (I & hprzz & hprzz & hprzz & I)
    return evens * odds

def Hd(b, dt):
    theta = b * dt * 2
    print(f"Hd with {theta}")
    return qh.eval(f"(Rz({theta:.20f}))(Rz(-1 * {theta:.20f}))" * 4)
    #return qh.eval(f"(Rx({theta}))" * 8)

def rzz(theta):
    return qh.eval(f"Exp((-1j * {theta:.20f} / 2) * (ZZ))")

def p_iSwap(theta):
    arr = np.array([[1, 0, 0, 0],
                    [0, np.cos(theta/2), -1j * np.sin(theta/2), 0],
                    [0, -1j * np.sin(theta/2), np.cos(theta/2), 0],
                    [0, 0, 0, 1]])
    return qh.QuantumElement(arr, qh.WaveFunctionTokens.OPERATOR)

In [254]:
T = 1
dt = 1 / 4

psi = qh.eval("|01010101>")
t = 0
while t < T:
    a = (t + dt) / T
    if a == 1:
        a = dt/2
    b = 1 - a
    psi = Hp(a, dt) * psi
    psi = Hd(b, dt) * psi
    t += dt


iswap with -0.125
Hd with 0.375
iswap with -0.25
Hd with 0.25
iswap with -0.375
Hd with 0.125
iswap with -0.0625
Hd with 0.4375


In [255]:
yHad = qh.QuantumElement( (1/np.sqrt(2)) * np.array([[1j, -1], [1, -1j]]), qh.WaveFunctionTokens.OPERATOR)

expectStrings = ['ZZIIIIII', 'IZZIIIII', 'IIZZIIII', 'IIIZZIII', 'IIIIZZII', 'IIIIIZZI', 'IIIIIIZZ']
xPsi = qh.eval("HHHHHHHH") * psi
gsXPsi = qh.eval("HHHHHHHH") * gs_psi
yPsi = (yHad & yHad & yHad & yHad & yHad & yHad & yHad & yHad) * psi
gsYPsi = (yHad & yHad & yHad & yHad & yHad & yHad & yHad & yHad) * gs_psi
zEnergy = 0
xEnergy = 0
yEnergy = 0
gsZEnergy = 0
gsXEnergy = 0
gsYEnergy = 0
for state in expectStrings:
    op = qh.eval(state)
    zEnergy += (psi.dagger() * op * psi).data
    xEnergy += (xPsi.dagger() * op * xPsi).data
    yEnergy += (yPsi.dagger() * op * yPsi).data
    gsZEnergy += (gs_psi.dagger() * op * gs_psi).data    
    gsXEnergy += (gsXPsi.dagger() * op * gsXPsi).data    
    gsYEnergy += (gsYPsi.dagger() * op * gsYPsi).data    
print(zEnergy)
print(xEnergy)
print(yEnergy)
print(zEnergy + xEnergy + yEnergy)

print(f"GS {gsZEnergy + gsYEnergy + gsXEnergy}")

(-5.519387091686517+0j)
(-2.608104597436939+0j)
(-2.6081045974369386+0j)
(-10.735596286560394+0j)
GS (-13.499730394751545+0j)
